## Imports

In [19]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

## Import data from PDF

In [20]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

## Pandas Manipulation

In [21]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [22]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

## Geocoder Setup and Execution

In [23]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [24]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [25]:
df['loc']

1     (8701 Collins Ave, Miami Beach, FL 33154, USA,...
2     (321 Ocean Dr, Miami Beach, FL 33139, USA, (25...
3     (18501 Collins Ave #4002, Sunny Isles Beach, F...
4     (1 Collins Ave Unit 307, Miami Beach, FL 33139...
5     (17475 Collins Ave UNIT 1001, Sunny Isles Beac...
6     (Trump Palace, 18101 Collins Ave, Sunny Isles ...
7     (488 NE 18th St #4012, Miami, FL 33132, USA, (...
8     (801 Brickell Key Blvd, Miami, FL 33131, USA, ...
9     (Jade at Brickell Bay, 1331 Brickell Bay Dr #4...
10    (465 Brickell Ave APT 4301, Miami, FL 33131, U...
Name: loc, dtype: object

In [26]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [27]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Eighty Seven Park\r8701 Collins Ave 1102/1106\...,9/13/2021,6/7/2022,165,"$19,152,000.00",5409,"$3,540.77",David Siddons,Douglas Elliman,Debra Golan PA,Douglas Elliman,Eighty Seven Park 8701 Collins Ave 1102/1106 M...,Eighty Seven Park,8701 Collins Ave 1102/1106 Miami Beach,"(8701 Collins Ave, Miami Beach, FL 33154, USA,...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0
2,321 Ocean Condo\r321 Ocean Dr BH200\rMiami Beach,4/12/2022,6/7/2022,16,"$9,000,000.00",4745,"$2,403.20",David Pulley PA,Douglas Elliman,Seth Feuer PA,"Compass Florida, LLC",321 Ocean Condo 321 Ocean Dr BH200 Miami Beach,,321 Ocean Condo 321 Ocean Dr BH200 Miami Beach,"(321 Ocean Dr, Miami Beach, FL 33139, USA, (25...","(25.7722958, -80.1319008, 0.0)",25.772296,-80.131901,0.0
3,Turnberry Ocean Club\r18501 Collins Ave 4002\r...,3/25/2022,6/6/2022,41,"$5,500,000.00",2821,"$1,949.66",Carolina Arenas,Brown Harris Stevens,Richard Goihman,Douglas Elliman,Turnberry Ocean Club 18501 Collins Ave 4002 Su...,Turnberry Ocean Club,18501 Collins Ave 4002 Sunny Isles Beach,"(18501 Collins Ave #4002, Sunny Isles Beach, F...","(25.9477873, -80.12022019999999, 0.0)",25.947787,-80.120220,0.0
4,One Ocean\r1 Collins Ave 307\rMiami Beach,9/8/2021,6/7/2022,209,"$4,550,000.00",3041,"$1,496.22",Anna Sherrill,One Sotheby's Intl Realty,Ana Teresa Rodriguez,Coldwell Banker Realty,One Ocean 1 Collins Ave 307 Miami Beach,One Ocean,1 Collins Ave 307 Miami Beach,"(1 Collins Ave Unit 307, Miami Beach, FL 33139...","(25.768079, -80.1339531, 0.0)",25.768079,-80.133953,0.0
5,Chateau Beach\r17475 Collins Ave 1001\rSunny I...,7/1/2021,6/6/2022,192,"$4,500,000.00",5210,"$1,063.83",Richard Goihman,Douglas Elliman,Darin Spencer Raymond PA,BHHS EWM Realty,Chateau Beach 17475 Collins Ave 1001 Sunny Isl...,Chateau Beach,17475 Collins Ave 1001 Sunny Isles Beach,"(17475 Collins Ave UNIT 1001, Sunny Isles Beac...","(25.9379684, -80.1207198, 0.0)",25.937968,-80.120720,0.0
6,Trump Palace\r18101 Collins Ave SPA209\rSunny ...,12/2/2021,6/7/2022,147,"$2,575,000.00",2681,"$1,057.49",Lana Bell,Douglas Elliman,Gustavo Strallnikoff,Dezer Platinum Realty LLC,Trump Palace 18101 Collins Ave SPA209 Sunny Is...,Trump Palace,18101 Collins Ave SPA209 Sunny Isles Beach,"(Trump Palace, 18101 Collins Ave, Sunny Isles ...","(25.9439355, -80.12004569999999, 0.0)",25.943935,-80.120046,0.0
7,Aria on the Bay\r488 NE 18th St 4012\rMiami,4/18/2022,6/9/2022,17,"$2,080,000.00",2018,"$1,030.72",Daniel Horowitz,BrokerNation Real Estate,Daniel Horowitz,BrokerNation Real Estate,Aria on the Bay 488 NE 18th St 4012 Miami,Aria on the Bay,488 NE 18th St 4012 Miami,"(488 NE 18th St #4012, Miami, FL 33132, USA, (...","(25.7935082, -80.18668810000001, 0.0)",25.793508,-80.186688,0.0
8,Courts Brickell Key Condo\r801 Brickell Key Bl...,4/5/2022,6/9/2022,20,"$2,025,000.00",1668,"$1,214.03",Lina Pagano Fals,Coldwell Banker Realty,Jorge Colona,SANCAS Realty,Courts Brickell Key Condo 801 Brickell Key Blv...,Courts Brickell Key Condo,801 Brickell Key Blvd PH03 Miami,"(801 Brickell Key Blvd, Miami, FL 33131, USA, ...","(25.7686611, -80.1866783, 0.0)",25.768661,-80.186678,0.0
9,Jade at Brickell\r1331 Brickell Bay Dr 4009\rM...,4/1/2022,6/8/2022,23,"$1,900,000.00",1730,"$1,098.27",Jordan Millman PA,West Avenue Realty,Laura Preuss Kuhne,"Compass Florida, LLC",Jade at Brickell 1331 Brickell Bay Dr 4009 Miami,Jade at Brickell,1331 Brickell Bay Dr 4009 Miami,"(Jade at Brickell Bay, 1331 Brickell Bay Dr #4...","(25.7599996, -80.1891629, 0.0)",25.760000,-80.189163,0.0
10,I0con Brickell\r465 Brickell Ave 4301\rMiami,4/4/2022,6/7/2022,6,"$1,810,000.00",1825,$991.78,Patricia Franco,The Keyes Company,Samuel Juarez,Luxe Properties,I0con Brickell 465 Brickell Ave 4301 Miami,I,0con Brickell 465 Brickell Ave 4301 Miami,"(465 Brickell Ave APT 4301, Miami,

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction Section
To be used if a name is off. Format for making corrections is as follows:

`df.at[index_num,'column_name']=('what you want inserted')`

In [30]:
df.at[2,'building_name']=('321 Ocean Condo')
df.at[2,'address_only']=('1 Collins Ave 307 Miami Beach')

In [32]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('June 5th - June 11th')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [33]:
m.save('index.html')